# Carga Gold - Fato de Cotação

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [22]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, year, month, current_timestamp

## Start Spark Session

In [23]:
# Inicializa uma SparkSession
builder = SparkSession.builder \
    .appName("Carga Fato Cotação") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado!\n")


✅ Spark 3.5.7 iniciado!



## Define Delta Table Paths

In [24]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_fato_cotacao = f"{base_gold_path}/fato_cotacao"

## Read Source Data

In [25]:
# Lê a tabela staging de cotação histórica
df_stg_cotacao_historica = spark.read.format("delta").load(delta_path_cotacao_historica)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cotacao_historica}")
print(f"Total de registros: {df_stg_cotacao_historica.count()}")

df_dim_ativo = spark.read.format("delta").load(delta_path_dim_ativo)

print(f"[SUCESSO] Leitura da tabela dimensão em: {delta_path_dim_ativo}")
print(f"Total de registros: {df_dim_ativo.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 5126223
[SUCESSO] Leitura da tabela dimensão em: D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_ativo_financeiro
Total de registros: 2262


## Transform Data

In [26]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(col("tp_mercado") == "010")

# Join LEFT (mantém todas as cotações mesmo sem correspondência na dimensão)
df_joined = (
    df_stg_cotacao_historica.alias("stg")
    .join(
        df_dim_ativo.alias("dim"),
        col("stg.cd_negociacao") == col("dim.cd_ativo"),
        "left"  # mantém as linhas da stg mesmo se não achar na dimensão
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_cotacao = (
    df_joined.select(
        col("stg.dt_pregao"),
        when(col("sk_ativo").isNull(), lit("-1")).otherwise(col("sk_ativo")).alias("sk_ativo"),
        col("vl_abertura"),
        col("vl_minimo"),
        col("vl_maximo"),
        col("vl_medio"),
        col("vl_ultimo_negocio"),
        col("qt_negocios_efetuados").alias("qt_negocio"),
        col("qt_total_titulos").alias("qt_titulo"),
        col("vl_total_titulos").alias("vl_volume"),
        current_timestamp().alias("ts_insercao"),
        year(col("dt_pregao")).alias("ano_pregao"),
        month(col("dt_pregao")).alias("mes_pregao")
    )
)

## Write Data

In [27]:

df_fato_cotacao.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .option("partitionOverwriteMode", "dynamic") \
    .partitionBy("ano_pregao", "mes_pregao") \
    .save(delta_path_fato_cotacao)

print("✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas")

✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas


## Stop Spark Session

In [28]:
# Encerra a SparkSession
spark.stop()